In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import gensim

In [2]:
df = pd.read_json("../data/processed_data/processed.json")
dti = pd.to_datetime(df[["year", "month", "day", "hour"]], errors="coerce")
df["time"] = dti
df

,paragraph,hour,day,month,year,filename,time
0,תם שימוע ראש הממשלה בינימין נתניהו לאחר עשר שע...,21,7,10,2019,/Users/sapir/PycharmProjects/hebnlp_project/da...,2019-10-07 21:00:00
1,גבר בן שלושים נורה ברחוב העלייה בלוד מצבו קשה ...,21,7,10,2019,/Users/sapir/PycharmProjects/hebnlp_project/da...,2019-10-07 21:00:00
2,הרמטכל רב אלוף אביב כוכבי מזהיר הערב לא נאפשר ...,21,7,10,2019,/Users/sapir/PycharmProjects/hebnlp_project/da...,2019-10-07 21:00:00
3,שר התחבורה בצלאל סמוטריץ הודיע בעמוד הפייסבוק ...,21,7,10,2019,/Users/sapir/PycharmProjects/hebnlp_project/da...,2019-10-07 21:00:00
4,נשיא ארצות הברית טראמפ הודיע יציב קווים אדומים...,21,7,10,2019,/Users/sapir/PycharmProjects/hebnlp_project/da...,2019-10-07 21:00:00
...,...,...,...,...,...,...,...
5141,בחירות אלפיים עשרים ואחת יושב ראש ימינה חבר הכ...,8,9,2,2021,/Users/sapir/PycharmProjects/hebnlp_project/da...,2021-02-09 08:00:00
5142,המחדל בנבטים במהלך אותרה הנקודה בגדר שממנה ככל...,8,9,2,2021,/Users/sapir/PycharmProjects/hebnlp_project/da...,2021-02-09 08:00:00
5143,הדיונים על פתיחת המסחר באמצעות הדרכון הירוק יי...,8,9,2,2021,/Users/sapir/PycharmProjects/hebnlp_project/da...,2021-02-09 08:00:00
5144,פרסום ראשון חיילת פנתה לרמטכל בדרישה פתח את כל...,8,9,2,2021,/Users/sapir/PycharmProjects/hebnlp_project/da...,2021-02-09 08:00:00


In [3]:
print(df.loc[0, "paragraph"])

תם שימוע ראש הממשלה בינימין נתניהו לאחר עשר שעות של דיונים בתיק אלף תיק המתנות היום נחתמו ארבעת ימי השימוע כעת על היועץ המשפטי לממשלה אביחי מנדלבליט להחליט האם להגיש אישום בארבעת התיקים יובל אראל מציין עורכי הדין יגישו סיכומים כתובים אחרי סוכות ומזכיר המשנה לפרקליט המדינה ליאת בן ארי נעדרה מן השימוע ביומיים האחרונים בשל חופשה משפחתית


# Tokenization

In [4]:
text_data = df.paragraph.values.tolist()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

data_words = list(sent_to_words(text_data))

# Bigrams and Trigrams

In [100]:
bigram = gensim.models.Phrases(data_words, min_count=50, threshold=1)
bigram_mod = bigram.freeze()
trigram = gensim.models.Phrases(bigram[data_words], min_count=50, threshold=1)
trigram_mode = trigram.freeze()

In [91]:
from tqdm import tqdm
def make_bigrams(texts):
    return [bigram[doc] for doc in tqdm(texts)]

def make_trigrams(texts):
    return [trigram[bigram_mod[doc]] for doc in tqdm(texts)]

In [92]:
bigram_data_words = make_bigrams(data_words)


100%|██████████| 5146/5146 [00:00<00:00, 10659.33it/s]


In [93]:
trigram_data_words = make_trigrams(bigram_data_words)


100%|██████████| 5146/5146 [00:00<00:00, 7207.95it/s]


In [94]:
stop_grams = ["גלי_צהל", "בגלי_צהל"]

def remove_stopgrams(texts, stop_grams):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_grams] for doc in tqdm(texts)]

In [95]:
processed_data = remove_stopgrams(trigram_data_words, stop_grams)


100%|██████████| 5146/5146 [00:00<00:00, 13558.11it/s]


# Create dictionary and Corpus